In [60]:
from datetime import datetime
import spotipy
import billboard
import re
import SpotipyAPITest as spot
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import train_test_split
import time
import pandas as pd
#import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials
cid = "b3c5b350c6cb43c2bdb403eb3b896eba"
secret = "fa1aa37ba9c943d3b4e2cadc8fd4818f"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


SyntaxError: invalid syntax (SpotipyAPITest.py, line 102)

In [ ]:
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

In [ ]:
def getTrackIDsFromAlbum(album_id):
    ids2 = []
    #playlist = sp.user_playlist(user, playlist_id)
    album = sp.album(album_id)
    for item in album['tracks']['items']:
        ids2.append(item['id'])
    return ids2

In [ ]:
def getTrackFeatures(id):
      meta = sp.track(id)
      features = sp.audio_features(id)

      # meta
      song_id = id
      name = meta['name']
      artist = meta['album']['artists'][0]['name']
      length = meta['duration_ms']
      popularity = meta['popularity']
      release_date = meta['album']['release_date']

      # features
      acousticness = features[0]['acousticness']
      danceability = features[0]['danceability']
      energy = features[0]['energy']
      instrumentalness = features[0]['instrumentalness']
      liveness = features[0]['liveness']
      loudness = features[0]['loudness']
      speechiness = features[0]['speechiness']
      tempo = features[0]['tempo']
      time_signature = features[0]['time_signature']
      valence = features[0]['valence']
      mode = features[0]['mode']
      weeks = 1
      chart_date = 1
      target = 1

      track = {"name" : name, "artist" : artist, "song_id" : song_id, "length" : length, "popularity" : popularity, 
               "danceability" : danceability, "acousticness" : acousticness, "energy" : energy, "instrumentalness" : instrumentalness, 
               "liveness" : liveness, "loudness" : loudness, "speechiness" : speechiness, "tempo" : tempo, "time_signature" : time_signature, 
               "valence" : valence, "mode" : mode, "weeks" : weeks, "chart_date" : chart_date, "release_date" : release_date, "target" : target}
      return track

In [ ]:
def getTrackIdFromBillboard(name, artist):
    results = sp.search(q="track:" + name + " artist:" + artist)
    items = results["tracks"]["items"]
    id = "NONE"
    if (len(items) > 0):
        id = items[0]["id"]
    return id

In [ ]:
def getChartDataFrame(chart, fullDataFrame):
    tracks = []
    for entry in chart.entries:
        print(entry.title)
        artists = re.split(' & | x | Featuring | With |, ', str(entry.artist).replace(entry.title + " ", ""))
        print(artists)
        id = getTrackIdFromBillboard(entry.title, artists[0])

        if (id != "NONE"):
            if (entry.artist in fullDataFrame.values) and (not any(dic['artist'] == entry.artist for dic in tracks)):
                fullDataFrame.loc[fullDataFrame['artist'] == entry.artist, 'weeks'] += 1
            if (id not in fullDataFrame.values) and (entry.isNew):
                track = spot.getTrackFeatures(id)
                track["chart_date"] = datetime.strptime(chart.date, '%Y-%m-%d').date()
                tracks.append(track)
    df = pd.DataFrame(tracks)
    fullDataFrame = fullDataFrame.append(df)
    return fullDataFrame

In [ ]:
def gatherBillboardData():
    chart = billboard.ChartData('hot-100')
    i = 0
    fullDataFrame = pd.DataFrame(columns = ["name", "artist", "song_id", "danceability", "energy",
                                                             "loudness", "mode", "speechiness", "acousticness",
                                                             "instrumentalness", "liveness", "valence", "tempo",
                                                             "duration_ms", "time_signature", "sections",
                                                             "target", "chart_date", "popularity", "release_date",
                                                             "weeks", "artist_popularity", "artist_followers",
                                                             "number_of_artists", "list_of_artists", "key"])
    start = time.time()
    while (chart.previousDate):
        #start = time.clock()
        print(chart)
        fullDataFrame = getChartDataFrame(chart, fullDataFrame)
        chart = billboard.ChartData('hot-100', chart.previousDate)
        #end = time.clock()
        #print("total time: " + str(end-start))
        i += 1
        if (i == 1): #was 960
            break
        if (i == 480):
            print("halveis")
    end = time.time()
    print("total time: " + str(end-start))
    return fullDataFrame
from pathlib import Path
data_folder = Path("CSV-files")
writeFile = "billboardv2.csv"
writePath = data_folder / writeFile

In [ ]:
df = gatherBillboardData()
#print(df)
df.to_csv(writeFile, index=False, header=True, columns=["name", "artist", "song_id", "danceability", "energy",
                                                             "loudness", "mode", "speechiness", "acousticness",
                                                             "instrumentalness", "liveness", "valence", "tempo",
                                                             "duration_ms", "time_signature", "sections",
                                                             "target", "chart_date", "popularity", "release_date",
                                                             "weeks", "artist_popularity", "artist_followers",
                                                             "number_of_artists", "list_of_artists", "key"])
#dataFrame.head(40)
#print(len(dataFrame))
#date1 = datetime.strptime('2020-10-17', '%Y-%m-%d').date()
#dataFrame.loc[(dataFrame['artist'] == '')]
